OIB radar reduction

Notebook to take block averges of radar derived thickness and bed hgtation to reduce data volume

In [1]:
### imports ##
import os, sys, math
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pyproj
import verde as vd

In [2]:
dat_path = 'F:/MARS/orig/xtra/OIB-AK/radar/nsidc/'
f = dat_path + '/IRARES2/IRARES2.gpkg'
print(os.path.isfile(f))

True


In [3]:
gdf = gpd.read_file(f)

In [4]:
gdf.head()

,fname,trace,lon,lat,hgt,srf_sample,srf_twtt,srf_hgt,bed_sample,bed_twtt,bed_hgt,ice_thickness,geometry
0,IRARES2_20160528-224052,0,-140.498316,59.793913,518.278004,218.0,0.000002,191.504238,NaN,NaN,NaN,NaN,POINT (-140.49832 59.79391)
1,IRARES2_20160528-224052,1,-140.498526,59.793948,518.566002,217.0,0.000002,193.291182,425.0,0.000004,17.620713,175.670469,POINT (-140.49853 59.79395)
2,IRARES2_20160528-224052,2,-140.498735,59.793983,518.851000,215.0,0.000002,196.574108,422.0,0.000004,21.748206,174.825901,POINT (-140.49874 59.79398)
3,IRARES2_20160528-224052,3,-140.498945,59.794018,519.129999,214.0,0.000002,198.352087,421.0,0.000004,23.526175,174.825912,POINT (-140.49894 59.79402)
4,IRARES2_20160528-224052,4,-140.499154,59.794053,519.399997,213.0,0.000002,200.121031,419.0,0.000004,26.139698,173.981333,POINT (-140.49915 59.79405)


In [5]:
# project
gdf = gdf.to_crs('epsg:3338')

In [6]:
# use verde to reduce along fname data 
reducer = vd.BlockReduce(np.mean, spacing=50)

fname = []
x = []
y = []
srf_z = []
bed_z = []
h = []

for fname_ in list(gdf.fname.unique()):
    (x_, y_), srf_ = reducer.filter((gdf[gdf['fname']==fname_].centroid.x.to_numpy(), gdf[gdf['fname']==fname_].centroid.y.to_numpy()), gdf[gdf['fname']==fname_].srf_hgt.to_numpy())
    (x_, y_), bed_ = reducer.filter((gdf[gdf['fname']==fname_].centroid.x.to_numpy(), gdf[gdf['fname']==fname_].centroid.y.to_numpy()), gdf[gdf['fname']==fname_].bed_hgt.to_numpy())
    (x_, y_), h_ = reducer.filter((gdf[gdf['fname']==fname_].centroid.x.to_numpy(), gdf[gdf['fname']==fname_].centroid.y.to_numpy()), gdf[gdf['fname']==fname_].ice_thickness.to_numpy())
    x.append(x_)
    y.append(y_)
    srf_z.append(srf_)
    bed_z.append(bed_)
    h.append(h_)
    fname.append([fname_ for i in range(len(x_))])

fname = np.hstack(fname)
x = np.hstack(x)
y = np.hstack(y)
srf_z = np.hstack(srf_z)
bed_z = np.hstack(bed_z)
h = np.hstack(h)

In [7]:
df = pd.DataFrame(
    {'fname': fname,
     'srf_z': srf_z,
     'bed_z': bed_z,
     'h': h,
     'x': x,
     'y': y})
     
gdf_out = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.x, df.y), crs='epsg:3338').to_crs('epsg:4326')

gdf_out = gdf_out.drop(columns=['x','y'])

gdf_out.to_file(dat_path + 'IRARES2_block_reduce.gpkg', driver="GPKG")